In [1]:
import os
import csv
import glob
import pandas as pd
from datetime import datetime
from tsfresh import extract_features

def read_activity_file(filepath, patient_id):
    data = []
    with open(filepath) as f:
        csv_reader = csv.reader(f, delimiter=";")
        next(csv_reader)
        for line in csv_reader:
            data.append([datetime.strptime(line[0], "%m-%d-%Y %H:%M").timestamp(), int(line[1].split(" ")[0])])
    data = pd.DataFrame(data, columns=["TIME", "ACC"])
    data["ID"] = patient_id
    return data

input_dir = "C:/Users/mhhas/Downloads/adhd-diagnosis-data/activity_data"
output_dir = os.path.join("preprocessed_data")

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

all_features = pd.DataFrame()  

for filepath in glob.glob(os.path.join(input_dir, "*.csv")):
    print("Reading %s" % filepath)
    patient_id = os.path.splitext(os.path.basename(filepath))[0]

    data = read_activity_file(filepath, patient_id)
    features = extract_features(data, column_id="ID", column_value="ACC", column_sort="TIME", n_jobs=0, show_warnings=False)

    all_features = pd.concat([all_features, features]) 

all_features.to_csv(os.path.join(output_dir, "all_features.csv"), index=False, sep=";")


Reading C:/Users/mhhas/Downloads/adhd-diagnosis-data/activity_data\patient_activity_01.csv


Feature Extraction: 100%|██████████| 1/1 [02:06<00:00, 126.77s/it]


Reading C:/Users/mhhas/Downloads/adhd-diagnosis-data/activity_data\patient_activity_02.csv


Feature Extraction: 100%|██████████| 1/1 [02:22<00:00, 142.84s/it]


Reading C:/Users/mhhas/Downloads/adhd-diagnosis-data/activity_data\patient_activity_03.csv


...

Feature Extraction: 100%|██████████| 1/1 [05:24<00:00, 324.18s/it]


Reading C:/Users/mhhas/Downloads/adhd-diagnosis-data/activity_data\patient_activity_19.csv


Feature Extraction: 100%|██████████| 1/1 [02:18<00:00, 138.97s/it]


Reading C:/Users/mhhas/Downloads/adhd-diagnosis-data/activity_data\patient_activity_20.csv


Feature Extraction:   0%|          | 0/1 [00:00<?, ?it/s]